In [ ]:
# !pip install transformers

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [59]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [82]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    ROBERTA_PATH = str(MODELS_PATH/'roberta-base_lm')
    TOKENIZER_PATH = str(MODELS_PATH/'roberta-base-0')
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'roberta-base'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [10]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [11]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [12]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [13]:
class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.ROBERTA_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(cfg.ROBERTA_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector)

In [14]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

### Evaluation and Prediction

In [15]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [16]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [17]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [18]:
def create_optimizer(model, base_lr=5e-5):
    named_parameters = list(model.named_parameters())
    
    regressor_param_start = 203
    roberta_parameters = named_parameters[:197]
    attention_parameters = named_parameters[199:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5
        if layer_num >= 133:
            lr = base_lr / 0.5
        elif layer_num >= 69:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [19]:
sample_optimizer = create_optimizer(sample_model)

In [20]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [21]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [22]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [23]:
class Trainer():
    def __init__(self, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                pred = self.model(input_ids, attention_mask)
                
                mse = mse_loss(pred.flatten(), target)
                
                mse.backward()
                
                self.optimizer.step()
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, self.model_path)
                    start = time.time()
                
                step += 1
        return best_val_rmse

In [24]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = kfold.split(train_df)

In [25]:
list_val_rmse = []

pbar = tqdm(enumerate(splits), total=cfg.NUM_FOLDS, position=0, leave=True)
for fold, (train_indices, val_indices) in pbar:
    pbar.set_description(f'Fold {fold}')
    model_path = cfg.MODEL_FOLDER/f"roberta-base_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
        
    optimizer = create_optimizer(model)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    trainer = Trainer(model, model_path, train_loader, val_loader, optimizer, scheduler = scheduler)
    list_val_rmse.append(trainer.train())
    
    tokenizer.save_pretrained(str(model_path.parent))
    del model
    gc.collect()
    
    if cfg.DEVICE == 'cuda':
        torch.cuda.empty_cache()
        
print("\nPerformance estimates:")
print(list_val_rmse)
print("Mean:", np.array(list_val_rmse).mean())

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 5.04 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9517 New best_val_rmse: 0.9517

16 steps took 3.76 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.759 New best_val_rmse: 0.759

16 steps took 3.81 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6664 New best_val_rmse: 0.6664

16 steps took 3.82 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.5823 New best_val_rmse: 0.5823

16 steps took 3.83 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5702 New best_val_rmse: 0.5702

16 steps took 3.84 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5391 New best_val_rmse: 0.5391

16 steps took 3.85 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6835 Still best_val_rmse: 0.5391 (from epoch 0)

16 steps took 3.85 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5267 New best_val_rmse: 0.5267

16 steps took 3.87 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.8343 Still best_val_rmse: 0.5267 (from epoch 0)

16 steps took 4.17 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5488 Still best_val_rmse: 0.5267 (from epoch 0)

16 step

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.65 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.047 New best_val_rmse: 1.047

16 steps took 3.92 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6807 New best_val_rmse: 0.6807

16 steps took 3.93 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6995 Still best_val_rmse: 0.6807 (from epoch 0)

16 steps took 3.89 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.604 New best_val_rmse: 0.604

16 steps took 3.94 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7154 Still best_val_rmse: 0.604 (from epoch 0)

16 steps took 3.89 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5419 New best_val_rmse: 0.5419

16 steps took 3.91 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5395 New best_val_rmse: 0.5395

16 steps took 3.89 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6336 Still best_val_rmse: 0.5395 (from epoch 0)

16 steps took 3.91 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6143 Still best_val_rmse: 0.5395 (from epoch 0)

16 steps took 4.14 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5099 New best_val_rmse: 0

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

...


16 steps took 4.55 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.06 New best_val_rmse: 1.06

16 steps took 3.86 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7764 New best_val_rmse: 0.7764

16 steps took 3.9 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7042 New best_val_rmse: 0.7042

16 steps took 3.91 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6157 New best_val_rmse: 0.6157

16 steps took 3.87 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5837 New best_val_rmse: 0.5837

16 steps took 3.88 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5696 New best_val_rmse: 0.5696

16 steps took 3.89 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5706 Still best_val_rmse: 0.5696 (from epoch 0)

16 steps took 3.87 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5473 New best_val_rmse: 0.5473

16 steps took 3.85 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5381 New best_val_rmse: 0.5381

16 steps took 4.2 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5134 New best_val_rmse: 0.5134

16 steps took 3.9 seconds
Epoch: 1 batch_num:

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.54 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9533 New best_val_rmse: 0.9533

16 steps took 3.86 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7232 New best_val_rmse: 0.7232

16 steps took 3.87 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6139 New best_val_rmse: 0.6139

16 steps took 3.88 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6167 Still best_val_rmse: 0.6139 (from epoch 0)

16 steps took 3.87 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6353 Still best_val_rmse: 0.6139 (from epoch 0)

16 steps took 3.88 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6616 Still best_val_rmse: 0.6139 (from epoch 0)

16 steps took 3.89 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6008 New best_val_rmse: 0.6008

16 steps took 3.91 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5916 New best_val_rmse: 0.5916

16 steps took 3.96 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5721 New best_val_rmse: 0.5721

16 steps took 4.25 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5232 New best_val_rmse: 0.5232

16 st

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.58 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.325 New best_val_rmse: 1.325

16 steps took 3.86 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8763 New best_val_rmse: 0.8763

16 steps took 3.86 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6236 New best_val_rmse: 0.6236

16 steps took 3.9 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7016 Still best_val_rmse: 0.6236 (from epoch 0)

16 steps took 3.86 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6466 Still best_val_rmse: 0.6236 (from epoch 0)

16 steps took 3.93 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5566 New best_val_rmse: 0.5566

16 steps took 3.91 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5631 Still best_val_rmse: 0.5566 (from epoch 0)

16 steps took 3.88 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6322 Still best_val_rmse: 0.5566 (from epoch 0)

16 steps took 3.91 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5647 Still best_val_rmse: 0.5566 (from epoch 0)

16 steps took 4.26 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5489 N

### Verify the model

In [27]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [28]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [29]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [38]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

Model 2


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

Model 3


...

Model 5


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

Model 6


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

CPU times: user 7.84 s, sys: 2.64 s, total: 10.5 s
Wall time: 8.73 s


In [40]:
tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = AutoTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}-{i}")
    tokenizers.append(tokenizer)

In [41]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
            cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
    return np.array(cls_embeddings)

In [42]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [43]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [52]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [60]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [61]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [66]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model.transformer_model if hasattr(inference_model, 'transformer_model') else inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.36550808806022805
Fold 1 (2361,) (472,)
rmse_score 0.3892447944996949
Fold 2 (2361,) (472,)
rmse_score 0.3575438931619624
Fold 3 (2361,) (472,)
rmse_score 0.32517562032047437
Fold 4 (2361,) (472,)
rmse_score 0.36159873705112916
Fold 5 (2361,) (472,)
rmse_score 0.37287193956391995
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3638330713907293
Fold 1 (2361,) (472,)
rmse_score 0.3805220156614261
Fold 2 (2361,) (472,)
rmse_score 0.3387056043447015
Fold 3 (2361,) (472,)
rmse_score 0.3514474564889266
Fold 4 (2361,) (472,)
rmse_score 0.35702522851613155
Fold 5 (2361,) (472,)
rmse_score 0.37597055255951656
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.39025906415086853
Fold 1 (2361,) (472,)
rmse_score 0.3756086186646978
Fold 2 (2361,) (472,)
rmse_score 0.34031009813519836
Fold 3 (2361,) (472,)
rmse_score 0.3793376712959961
Fold 4 (2361,) (472,)
rmse_score 0.3635410022501858
Fold 5 (2361,) (472,)
rmse_score 0.38556034255063004
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3444165671558248
Fold 1 (2361,) (472,)
rmse_score 0.35942624057248
Fold 2 (2361,) (472,)
rmse_score 0.34570919000547357
Fold 3 (2361,) (472,)
rmse_score 0.31404498682049775
Fold 4 (2361,) (472,)
rmse_score 0.3371218351046066
Fold 5 (2361,) (472,)
rmse_score 0.3450690014221917
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3744261922049248
Fold 1 (2361,) (472,)
rmse_score 0.38088212096522267
Fold 2 (2361,) (472,)
rmse_score 0.34247634124874293
Fold 3 (2361,) (472,)
rmse_score 0.3431119256618516
Fold 4 (2361,) (472,)
rmse_score 0.37095399450067695
Fold 5 (2361,) (472,)
rmse_score 0.38254104366823705
FINAL RMSE score 0.36047477459990496
CPU times: user 2min 14s, sys: 1.91 s, total: 2min 16s
Wall time: 2min 15s


In [67]:
final_kernel_predictions_means

[[0.005153221483623596],
 [0.003964462237853874],
 [0.006731210171453145],
 [0.011433733987456354],
 [-0.0014762375862074479]]

In [68]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [69]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.19978976, 0.19989238, 0.19834089, 0.20270617, 0.1992708 ])

In [70]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [71]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.9893307630009803)

In [72]:
final_scores_flat

array([-0.69439869, -0.37979068, -0.36879958, -2.49210601, -1.7894283 ,
       -1.51868395,  0.31567297])

In [73]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(0.02999035576716491, 0.005998071153432982)

In [74]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.664408
1,f0953f0a5,-0.349800
2,0df072751,-0.338809
3,04caf4e0c,-2.462116
4,0e63f8bea,-1.759438
5,12537fe78,-1.488694
6,965e592c0,0.345663


### Prepare Packaging

In [83]:
cfg.model_name

'roberta-base'

In [84]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [85]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/roberta-base/best')

In [87]:
cfg.NUM_FOLDS

6

In [90]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [91]:
bestmodels

[PosixPath('/home/commonlit/models/roberta-base_1'),
 PosixPath('/home/commonlit/models/roberta-base_2'),
 PosixPath('/home/commonlit/models/roberta-base_3'),
 PosixPath('/home/commonlit/models/roberta-base_4'),
 PosixPath('/home/commonlit/models/roberta-base_5'),
 PosixPath('/home/commonlit/models/roberta-base_6')]

In [92]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    best_model_file = f'{best_model}/model_{i + 1}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}-{i}/tokenizer.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}-{i}/vocab.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'vocab.json')

        config_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}-{i}/config.json'))
        assert config_json.exists()
        copyfile(config_json, tokenizer_path/'config.json')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [93]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/roberta-base/best_models.zip'

In [94]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip  dataset-metadata.json  lm  lm.zip  roberta-base.yaml


In [95]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/roberta-base.yaml': No such file or directory


In [96]:
transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [97]:
!du -h {MODELS_PATH/cfg.model_name}/*

2.1M	/home/commonlit/models/roberta-base/best/tokenizer-0
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-1
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-2
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-3
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-4
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-5
2.9G	/home/commonlit/models/roberta-base/best
2.7G	/home/commonlit/models/roberta-base/best_models.zip
4.0K	/home/commonlit/models/roberta-base/dataset-metadata.json
476M	/home/commonlit/models/roberta-base/lm
442M	/home/commonlit/models/roberta-base/lm.zip
4.0K	/home/commonlit/models/roberta-base/roberta-base.yaml


In [98]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/roberta-base/lm.zip'

In [99]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/roberta-base/dataset-metadata.json


In [100]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [101]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [102]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}-light').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}-light')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-roberta-base-light",
  "id": "gilfernandes/commonlit-roberta-base-light",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [103]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [104]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 2.60G/2.60G [04:31<00:00, 10.3MB/s]
Upload successful: best_models.zip (3GB)
Starting upload for file roberta-base.yaml
100%|███████████████████████████████████████████| 114/114 [00:04<00:00, 24.4B/s]
Upload successful: roberta-base.yaml (114B)
Starting upload for file lm.zip
100%|████████████████████████████████████████| 442M/442M [00:54<00:00, 8.57MB/s]
Upload successful: lm.zip (442MB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gilfernandes/commonlit-roberta-base-light


In [ ]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with normal distribution by bin and extra pre-training" -d

In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)